# Aufgabe 4: Faltungstheorem
Das Faltungstheorem
\begin{align}
 F * G &= \mathcal{F}^{-1}(\mathcal{F}(F) \cdot \mathcal{F}(G))
\end{align}

besagt, dass eine Faltung im Ortsraum äquivalent zu einer Multiplikation im Frequenzraum ist.
Prüfen Sie die Gültigkeit dieses Theorems an mindestens zwei praktischen Beispielen mit selbst gewählten Filtermasken!
Vergleichen und analysieren Sie sowohl die Ergebnisse als auch die Rechenzeiten im Hinblick auf verschiedene Filtergrößen!

## 0. Pfade, Pakete etc.

In [ ]:
import glob
import imageio
import numpy as np
import scipy.ndimage
import math

%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
image_filter = 'Bilder/*.jpg'

## 1. Definition der Faltungsmaske
Definieren Sie hier die zu prüfenden Faltungsmaske `A` und `B`.

In [ ]:
m = 11
sigma = m / 5

offset = (m - 1) // 2
A = np.asarray([
    [
        np.exp(-(((cx-offset)**2) + ((cy-offset)**2)) / (2 * (sigma ** 2)))
        for cx in range(m)
    ] for cy in range(m)
])
A /= np.sum(A)

B = np.array([
    [-1, 0, 1],
    [-2, 0, 2],
    [-1, 0, 1]
])

## 2. Laden und Normalisieren des Bildes

In [ ]:
image_path = np.random.choice(glob.glob(image_filter))
image = imageio.imread(image_path)

In [ ]:
image = image.astype(np.float32)
image -= image.min()
image /= image.max()

## 3. Berechnung der Fouriertransformation
Setzen Sie hier ihre Lösung aus der vorigen Aufgabe ein:

In [ ]:
image_transformed = np.fft.fft2(image)

Berechnen Sie nun die Fouriertransformation der Faltungsmasken. Achten Sie darauf, dass das Ergebnis dieser Operation dieselbe Größe hat wie `image_transformed`!

In [ ]:
A_transformed = np.fft.fft2(A, image.shape)
B_transformed = np.fft.fft2(B, image.shape)

## 4. Filterung

Definieren Sie nun eine Funktion `ex3_filter_in_freq_domain`, die einen Filter im Frequenzbereich auf ein bereits fouriertransformiertes Bild anwendet. Beachten Sie das Faltungstheorem!

In [ ]:
def ex3_filter_in_freq_domain(spectrum, transformed_filter):
    return spectrum * transformed_filter

Das transformierte Bild (Spektrum) wird nun gefiltert.

In [ ]:
%time image_transformed_filtered_A = ex3_filter_in_freq_domain(image_transformed, A_transformed)
%time image_transformed_filtered_B = ex3_filter_in_freq_domain(image_transformed, B_transformed)

## 5. Inverse Filterung
Das veränderte Spektrum soll nun in den Ortsbereich zurücktransformiert werden. Verwenden Sie dazu die entsprechenden Funktionen des Paketes `numpy.fft`.

In [ ]:
image_filtered_A = np.fft.ifft2(image_transformed_filtered_A).real
image_filtered_B = np.fft.ifft2(image_transformed_filtered_B).real

Vergleichen Sie nun das gefilterte Bild mit dem Originalbild:

In [ ]:
plt.figure('Convolution: image comparison', figsize=(12, 6))
plt.subplot(1,3,1, title='Original Image')
plt.imshow(image, cmap='gray', vmin=0, vmax=1)
plt.subplot(1,3,2, title='Gauss-Filter')
plt.imshow(image_filtered_A, cmap='gray')
plt.subplot(1,3,3, title='Sobel-Filter')
plt.imshow(image_filtered_B, cmap='gray', vmin=0, vmax=1)
plt.show()

## 6. Vergleich mit regulärer Faltung
Im Folgenden wird das Bild mittels der Bibliotheksfunktion `scipy.ndimage.filters.convolve` mit `A` und `B` gefaltet.

In [ ]:
%time image_convolved_A = scipy.ndimage.filters.convolve(image, A, mode='constant')
%time image_convolved_B = scipy.ndimage.filters.convolve(image, B, mode='constant')

Vergleichen Sie `image_convolved` mit `image_filtered`, indem Sie die Bilder nebeneinander anzeigen:

In [ ]:
plt.figure(figsize=(12, 12))
plt.subplot(2,2,1, title='Gauss-Filter')
plt.ylabel('Convolved')
plt.imshow(image_convolved_A, cmap='gray')
plt.subplot(2,2,2, title='Sobel-Filter')
plt.imshow(image_convolved_B, cmap='gray')
plt.subplot(2,2,3)
plt.ylabel('Fourier Transform')
plt.imshow(image_filtered_A, cmap='gray')
plt.subplot(2,2,4)
plt.imshow(image_filtered_B, cmap='gray')
plt.show()

Eigentlich sollten die Bilder gleich aussehen, wie man sieht besteht aber an den rändern der bilder ein leichter Unterschied.

### Welche Aussage lässt sich hinsichtlich der Rechenzeit der Faltung mittels Fourier-Transformation im Vergleich zur regulären Faltung treffen?

Für kleine Werte ist der Unterschied nicht gerade groß, aber für größere Werte stellt man fest, dass sich die Fourier-Transform im vergleich zur Convolution linear verhält und Convolution eine quadratische Laufzeit aufweist, welcher aber von der größe der Maske abhängt.